In [9]:
from langchain_openai.llms import OpenAI

In [10]:
llm = OpenAI()

In [11]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("""
    Responda a seguinte pergunta do usuário:
    {pergunta}
"""
)

In [12]:
print( prompt_template.format(pergunta="O que é um SaaS?"))


    Responda a seguinte pergunta do usuário:
    O que é um SaaS?



In [13]:
prompt_template = PromptTemplate.from_template("""
    Responda a seguinte pergunta do usuário em até {n_palavras} palavras:
    {pergunta}
"""
)

In [14]:
print( prompt_template.format(pergunta="O que é um SaaS?", n_palavras=15))


    Responda a seguinte pergunta do usuário em até 15 palavras:
    O que é um SaaS?



In [16]:
prompt_template = PromptTemplate.from_template("""
    Responda a seguinte pergunta do usuário em até {n_palavras} palavras:
    {pergunta}
""",
partial_variables={"n_palavras": 10}
)

In [17]:
print( prompt_template.format(pergunta="O que é LangChain?"))


    Responda a seguinte pergunta do usuário em até 10 palavras:
    O que é LangChain?



### Utilizando múltiplos prompts

In [42]:
from langchain.prompts import PromptTemplate

template_word_cont = PromptTemplate.from_template("""
    Responda a seguinte pergunta do usuário em até {n_palavras} palavras:
    {pergunta}
"""
)

template_line_count = PromptTemplate.from_template("""
    Responda a seguinte pergunta do usuário em até {n_linhas} linhas:
    {pergunta}
"""
)

template_idioma = PromptTemplate.from_template("""
    Responda a seguinte pergunta do usuário em {idioma}.
"""
)

template_final = (template_word_cont + template_line_count + template_idioma +
"    Responda a pergunta seguindo as instruções {pergunta}."
)

In [24]:
print(template_final)

input_variables=['idioma', 'n_linhas', 'n_palavras', 'pergunta'] template='\n    Responda a seguinte pergunta do usuário em até {n_palavras} palavras:\n    {pergunta}\n\n    Responda a seguinte pergunta do usuário em até {n_linhas} linhas:\n    {pergunta}\n\n    Responda a seguinte pergunta do usuário em {idioma}.\nResponda a pergunta seguindo as instruções {pergunta}.'


In [43]:
prompt_final = template_final.format(n_palavras=15, n_linhas=2, idioma="inglês", pergunta="O que é LangChain?")
llm.invoke(prompt_final)

'\nLangChain is a blockchain platform that provides a decentralized environment for building and running smart contracts. It aims to revolutionize the way we interact and transact with each other.'

In [44]:
print(prompt_final)


    Responda a seguinte pergunta do usuário em até 15 palavras:
    O que é LangChain?

    Responda a seguinte pergunta do usuário em até 2 linhas:
    O que é LangChain?

    Responda a seguinte pergunta do usuário em inglês.
    Responda a pergunta seguindo as instruções O que é LangChain?.
